In [2]:
from keras.models import  load_model
from keras.utils import np_utils
import glob
import numpy as np
import os
import sed_eval
import dcase_util
import pickle
#%run shared_functions.ipynb

from PIL import Image

Using TensorFlow backend.


In [3]:

def getStackFeatMat(scene):
    with open(scene, "rb" ) as scenesample:
        l,fv = pickle.load(scenesample)
    return fv,l

def reduceLabels(labels):
    r,c=labels.shape 
    npOfLabels=np.zeros(c)
    for j in range(0,c):
        if np.sum(labels[:,j])>0:
            npOfLabels[j]=1
    return npOfLabels


def fileLists():
    trainlist=[]
    validationlist=[]
    testlist=[]
    evalSetupFiles='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\evaluation_setup\\*.txt'
    txtfilelist=glob.glob(evalSetupFiles)
    for txt in txtfilelist:
        if '_location' not in txt:
            continue
        if 'train' in txt:
            continue
        with open(txt,'r') as evaltxtfile:
            for line in evaltxtfile.readlines():
                line=line.strip().split('\t')[0]
                if 'train' in txt:
                    trainlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                elif 'test' in txt:
                    testlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
                else:
                    validationlist.append(line.replace('audio','logMelSpec').replace('.wav','_aggScenes.pckl').replace('/','\\'))
    print('validationfiles: ', str(len(validationlist)))
    print('testfiles: ', str(len(testlist)))
    return validationlist,testlist

labelRef={'inside-barcelona':0, 'outside-barcelona':1, 'transport-barcelona':2, 
          'inside-helsinki':3, 'outside-helsinki':4, 'transport-helsinki':5,
          'inside-london':6, 'outside-london':7,'transport-london':8,
          'inside-paris':9, 'outside-paris':10,'transport-paris':11,
          'inside-stockholm':12, 'outside-stockholm':13,'transport-stockholm':14,
          'inside-vienna':15, 'outside-vienna':16,'transport-vienna':17}

nb_classes=len(labelRef.keys())
def getData(flist):
    pth='..\\..\\CASAdatasets\\DCASE18_ASCT1\\TUT-urban-acoustic-scenes-2018-development\\'
    X_=np.zeros(((len(flist)),128,431))
    Y_=np.zeros(((len(flist)),nb_classes))
    for i,tfile in enumerate(flist):
        with open(pth+tfile, "rb" ) as scenesample:
            fv=pickle.load(scenesample)
        X_[i,:,:]=fv
        scene=''.join(tfile.split('\\')[1]).split('-')[0]
        location=''.join(tfile.split('\\')[1]).split('-')[1]
        if scene in ['airport','shopping_mall','metro_station']:
            scene='inside-'+location
        elif scene in ['street_pedestrian','public_square','street_traffic','park']:
            scene='outside-'+location
        else:
            scene='transport-'+location
        Y_[i,labelRef[scene]]=1
    return X_, Y_
validationlist,testlist=fileLists()
X_v, Y_val = getData(validationlist)
X_te, Y_test = getData(testlist)
print('data obtained')
X_val=np.expand_dims(X_v,axis=3)
X_test=np.expand_dims(X_te,axis=3)

validationfiles:  2518
testfiles:  2518
data obtained


In [32]:
#checking not negative values in Y_test
for row in Y_test:
    for v in row:
        if v < 0:
            print('found')

In [12]:
#split X_test into 18. One per pair.
X_test_subsets=np.zeros((6,480,128,431,1))
Y_test_subsets=np.zeros((6,480,18))
pos=np.zeros((6),dtype=np.int32)
#print(Y_test.shape, X_test.shape)
for i,xt_sample in enumerate(X_test):
    for k,j in enumerate(Y_test[i]):
        j=int(j)
        if j==1: #ground truth say yes
            # check the city (three possible k's to each one city)
            if k in [0,1,2]:
                idx=0
            elif k in [3,4,5]:
                idx=1
            elif k in [6,7,8]:
                idx=2
            elif k in [9,10,11]:
                idx=3
            elif k in [12,13,14]:
                idx=4
            else:
                idx=5            
            X_test_subsets[idx,pos[idx],:,:,:]=xt_sample
            Y_test_subsets[idx,pos[idx],:]=Y_test[i]
            pos[idx]=pos[idx]+1

#print(i)
#path='models\\best_model.hdf5'
#model=load_model(path)
#nb_classes=6
#use pos to remove empty testfiles 
#print(Y_test_subsets[0:pos[0],:])

cities=['barcelona','helsinki','london','paris','stockholm','vienna']
for i,testfiles in enumerate(X_test_subsets):
    print(cities[i])
    t_files=testfiles[0:pos[i],:,:,:] #remove empties
    y_labs=Y_test_subsets[i,0:pos[i],:]
    evaluation=model.evaluate(t_files, y_labs, verbose=0,batch_size=8)
    print(model.metrics_names)
    print(evaluation)
print("FIIIINESSSHED :-D")

6
barcelona
['loss', 'acc']
[2.2518668376916162, 0.5]
helsinki
['loss', 'acc']
[2.5150652858946057, 0.39583333333333331]
london
['loss', 'acc']
[1.880331775546074, 0.47499999999999998]
paris
['loss', 'acc']
[1.2389572250835394, 0.66894977141733036]
stockholm
['loss', 'acc']
[0.81918879552969281, 0.77426636568848761]
vienna
['loss', 'acc']
[3.305106631802841, 0.27323943661971833]
FIIIINESSSHED :-D


In [6]:
#evaluate performance of each model
path='models\\groupedScenePairs.hdf5'
model=load_model(path)
nb_classes=6
#predictions=np.zeros((Y_test.shape))
#print(predictions.shape)
#print('**')
#predictions=model.predict(X_test, verbose=1, batch_size=8) #
#asc(predictions,'test')
#print("FIIIINESSSHED :-D")

In [7]:

evaluation=model.evaluate(X_test, Y_test, verbose=0,batch_size=8)
print(model.metrics_names)
print(evaluation)

['loss', 'acc']
[1.9464396030062883, 0.52303415410238374]


['loss', 'acc']
[1.7743403799861457, 0.21247021447958883]


In [36]:
def hv(preds):
    scene=[]
    s,r=preds.shape
    hv=np.zeros((s,r))
    for i in range(0,s):
        for j in range(0,r):
             if preds[i,j]>globalthreshold:
                hv[i,j]=1
    return hv

print(predictions.shape)
globalthreshold=0.2
test_scenes=hv(predictions)
print(test_scenes.shape)
print(sum(test_scenes))

def asc(scene,evaluate):
    correct=0  
    for idx,s in enumerate(scene):
        match=False
        if sum(s)>1:
            print('WARNING:')
            print(s)
        for j,element in enumerate(s):
            if element==Y_test[idx][j] and element==1:
                match=True
        if match:
            correct=correct+1
    
    print('*** ASC:')
    print('correct scenes: ' +str(correct)+ '\n Accuracy:')
    print(correct/len(Y_test)) 
    return correct/len(Y_test)

acc=asc(test_scenes,'test')

(2518, 6)
(2518, 6)
[ 529.  143.  333.  555.  589.  630.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 0.  0.  0.  1.  1.  0.]
[ 1.  1.  0.  0.  1.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 0.  0.  0.  1.  1.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  0.  0.  1.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  

[ 0.  0.  0.  0.  1.  1.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  0.  1.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  1.  0.  0.  0.]
[ 0.  0.  0.  0.  1.  1.]
[ 1.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  0.  1.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 0.  0.  1.  1.  1.  0.]
[ 0.  0.  1.  0.  1.  0.]
[ 0.  0.  1.  0.  1.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  1.  1.  0.  0.]
[ 1.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  1.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  0.  1.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  0.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  1.  0.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 0.  0.  1.  1.  0.  0.]
[ 1.  0.  1.

In [ ]:
import itertools
def plot_confusion_matrix(mat, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          output = None):
    
    size = 3.5 + len(classes) * 0.5
    plt.figure(figsize=(size, size))
    plt.imshow(mat, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, horizontalalignment='right')
    plt.yticks(tick_marks, classes)

    fmt = '.0f' 
    thresh = mat.max() / 2.
    for i, j in itertools.product(range(mat.shape[0]), range(mat.shape[1])):
        plt.text(j, i, format(mat[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="white" if mat[i, j] > thresh else "black")

    plt.tight_layout()
    #plt.ylabel('True label')
    #plt.xlabel('Predicted label')
    
    if output is not None:
        plt.savefig(output, bbox_inches='tight')
        
#strip out non-supermarket events

scene_mat=np.random.randint(9, size=(6, 6)) #need actual class values for this!
classes=[labelRef.keys()]
plot_confusion_matrix(scene_mat, classes, title='', output='scenes.pdf')
